<a href="https://colab.research.google.com/github/zsevall/DSAI545/blob/main/TurkishELIZA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Original code of ELIZA with small updates

import string
import re
import random
import unicodedata # for normalize_text function

class Eliza:
    def __init__(self):
        # re.compile(x[0], re.IGNORECASE): Compiles the first element (x[0], a regex pattern) into a case-insensitive regular expression
        # map(...) applies this transformation to all elements in gPats, creating a list of regex patterns
        self.keys = list(map(lambda x: re.compile(x[0], re.IGNORECASE), gPats))
        self.values = list(map(lambda x: x[1], gPats))  #Extracts the second part (x[1], the response list) from gPats and stores it

    def normalize_text(self, text): # Turkish English keyboard mistake handling
        # Normalize text using unicodedata (e.g., remove accents)
        text = unicodedata.normalize('NFKC', text)  # Normalize Unicode characters
        text = text.lower()  # Convert to lowercase for consistency
        turkish_corrections = {
            # Basic emotional states and common expressions
            "uzgunum": "üzgünüm",
            "mutluyum": "mutluyum",
            "iyiyim": "iyiyim",
            "kotuyum": "kötüyüm",
            "hakliyim": "haklıyım",
            "hastayim": "hastayım",
            "yorgunum": "yorgunum",
            "yalnizim": "yalnızım",
            "uzgundum": "üzgündüm",
            "kizginim": "kızgınım",
            "ofkeliyim": "öfkeliyim",
            "sinirlyim": "sinirliyim",
            "heyecanliyim": "heyecanlıyım",
            "saskinim": "şaşkınım",
            "umutluyum": "umutluyum",
            "umitsizim": "umutsuzim",
            "endiseliyim": "endişeliyim",
            "kaygiliyim": "kaygılıyım",
            "memnunum": "memnunum",
            "mutsuzum": "mutsuzum",
            "sikildim": "sıkıldım",

            # Other common Turkish words with special chars
            "cikis": "çıkış",
            "nasil": "nasıl",
            "bilmiyorum": "bilmiyorum",
            "degilim": "değilim",
            "tesekkur": "teşekkür",
            "lutfen": "lütfen",
            "dusunuyorum": "düşünüyorum",
            "simdi": "şimdi",
            "belki": "belki",
            "cunku": "çünkü"
        }
        words = text.split()
        corrected_words = [turkish_corrections.get(word, word) for word in words]
        return " ".join(corrected_words)

    def translate(self, text, vocabulary):
        words = text.lower().split()  #Converts text to lowercase and splits it into words
        for i in range(len(words)):
            if words[i] in vocabulary: #In Python 3, it is not necessary to store vocabulary.keys() we can check words directly from vocabulary
                words[i] = vocabulary[words[i]]   # If a word exists in the vocabulary dictionary, it replaces it with its corresponding value.
        return ' '.join(words)

    """ Example use of translate function>>>
          text = "I am happy"
          words = text.lower().split()  # ['i', 'am', 'happy']
          output = "you are happy"
    """

    def respond(self, text):
        text = self.normalize_text(text)
        for i in range(len(self.keys)): # Loops through self.keys (regex patterns),
            match = self.keys[i].match(text) # Checks if the user input text matches any pattern
            if match: # if any match, responds randomly from predefined answers
                resp = random.choice(self.values[i])
                pos = resp.find('%')  # searches for % in the response: %1 refers to the first captured group from the regex match. %2 would refer to the second captured group, and so on.
                # It finds placeholders (%1, %2, etc.) in a response string.
                # It replaces them with captured groups from the matched user input.
                # It uses gReflections to modify certain words (e.g., "I" → "you").
                while pos > -1:
                    num = int(resp[pos+1:pos+2])  # pos+1:pos+2 extracts the number right after %, num stores them as integer
                    resp = resp[:pos] + self.translate(match.group(num), gReflections) + resp[pos+2:]  # Replace %1 with Matched Group, gReflections modify words dynamically (e.g., "I" → "you").
                    pos = resp.find('%')
                if resp[-2:] == '?.': resp = resp[:-2] + '.'
                if resp[-2:] == '??': resp = resp[:-2] + '?'
                return resp
        return  "Bunu tam olarak anlayamadım. Daha farklı anlatabilir misiniz?"  # If % is not found, find() returns -1, and the loop doesn't execute, it is generic answer

    """ Example use of respond function>>>
          gPats = [
                [r'I need (.*)',
                [  "Why do you need %1?",
                    "Would it really help you to get %1?",
                    "Are you sure you need %1?"]],
                ]
          user_input = "I need a break"
          pattern: r'I need (.*)'
  	      Input: "I need a break"
          Regex Match: "I need a break"
          Captured Group (match.group(1)): "a break"
          Random choice of response>> "Why do you need %1?"
          Output: Why do you need a break?
    """

gReflections = {
    "ben"   : "siz",
    "beni"  : "sizi",
    "benim" : "sizin",
    "bana"  : "size",
    "benden": "sizden",
    "benimki" : "sizinki",
    "siz"   : "ben",
    "sizi"  : "beni",
    "sizin" : "benim",
    "size"  : "bana",
    "sizden": "benden",
    "sizinki" : "benimki",
    "istiyorum": "istiyorsunuz",
    "istiyorsunuz": "istiyorum",
    "düşünüyorum": "düşünüyorsunuz",
    "düşünüyorsunuz": "düşünüyorum",
    "hissediyorum": "hissediyorsunuz",
    "hissediyorsunuz": "hissediyorum",
    "kendimi": "kendinizi"
}

# gPats is list of tuples containing regex patterns and responses
# inspired by Original ELIZA
gPats = [
    [r'(mutlu|iyi|kötü|üzgün|yorgun|yalnız|kızgın|öfkeli|sinirli|heyecanlı|şaşkın|umutlu|umutsuz|endişeli|kaygılı|memnun|mutsuz)(y|)(im|ım|um|üm)$',
    [  "Ne zamandan beri %1sünüz?",
       "%1 olmanızın sebebi ne olabilir?",
       "%1 hissetmek sizin için nasıl bir deneyim?",
       "Kendinizi %1 hissettiğinizde genellikle ne yaparsınız?"]],

    [r'Merhaba(.*)',
    [  "Merhaba! Bugün nasılsınız?",
       "Selam! Bugün nasıl hissediyorsunuz?",
       "Merhaba, size nasıl yardımcı olabilirim?"]],

    [r'(.*)üzgünüm',
    [  "Bazen üzülmek doğaldır.",
       "Üzgün hissetmenize ne sebep oldu?",
       "Bunu biraz daha anlatmak ister misiniz?"]],

    [r'(.*) hissediyorum',
    [  "%1 hissetmek nasıl bir şey?",
       "Ne zamandan beri %1 hissediyorsunuz?",
       "%1 hissetmenize ne sebep oldu?"]],

    [r'(.*) istiyorum',
    [  "Neden %1 istiyorsunuz?",
       "Gerçekten %1 ihtiyacınız var mı?",
       "%1 sizin için ne ifade ediyor?"]],

    [r'(.*) yapamıyorum',
    [  "Neden %1 yapamayacağınızı düşünüyorsunuz?",
       "Belki de denerseniz %1 yapabilirsiniz.",
       "%1 yapabilmek için neye ihtiyacınız var?"]],

    [r'(b|B)en (.*)',
    [  "%2 olmanız hakkında ne düşünüyorsunuz?",
       "Kendinizi %2 olarak görmeniz nasıl hissettiriyor?",
       "%2 olduğunuzu düşündüğünüzde neler hissediyorsunuz?"]],

    [r'Siz (.*) misiniz\??',
    [  "Bunun bir önemi var mı?",
       "Beni %1 olarak düşünmeyi mi tercih edersiniz?",
       "Belki %1 olabilirim, belki değilim. Siz ne düşünüyorsunuz?"]],

    [r'(.*) nedir\??',
    [  "Neden %1 merak ediyorsunuz?",
       "%1 cevabını bilmek size nasıl yardımcı olur?",
       "Siz %1 hakkında ne düşünüyorsunuz?"]],

    [r'Nasıl (.*)\??',
    [  "Bunu nasıl düşündüğünüzü açıklayabilir misiniz?",
       "%1 konusunu biraz daha açabilir misiniz?",
       "Sizce nasıl %1?"]],

    [r'Neden (.*) yapmıyorsunuz\??',
    [  "Sizce gerçekten %1 yapmıyor muyum?",
       "Belki bir gün %1 yaparım.",
       "Bunu yapmamı ister misiniz?"]],

    [r'Çünkü (.*)',
    [  "Başka hangi sebepler olabilir?",
       "Eğer %1 ise, başka ne doğru olabilir?",
       "%1 sizin için ne ifade ediyor?"]],

    [r'Evet',
    [  "Evet derken neyi kastettiğinizi açıklayabilir misiniz?",
       "Peki, biraz daha anlatır mısınız?",
       "Anladım, devam edin lütfen."]],

    [r'(H|h)ayır',
    [  "Neden hayır diyorsunuz?",
       "Hayır demenizin özel bir nedeni var mı?",
       "Bu konu hakkında başka düşünceleriniz var mı?"]],

    [r'(B|b)ilmiyorum',
    [  "Peki, daha önce bu konuda düşündünüz mü?",
       "Bunu öğrenmek ister misiniz?",
       "Emin olmadığınız başka şeyler var mı?"]],

    [r'(.*) düşünüyorum',
    [  "Neden %1 düşünüyorsunuz?",
       "%1 düşünmek sizi nasıl hissettiriyor?",
       "Bu düşünce ne zamandan beri aklınızda?"]],

    [r'(.+?)(y|)(im|ım|um|üm)$',
    [ "Neden kendinizi %1 hissediyorsunuz?",
      "Ne zamandan beri böyle hissediyorsunuz?",
      "%1 kelimesiyle tam olarak neyi kastediyorsunuz?",
      "Bu durumda size nasıl yardımcı olabilirim?"]],
]

def command_interface():
    print('Turkish ELIZA\n---------')
    print('Terapi Botu\n---------')
    print('ELIZA, Rogerian tarzda bir psikoterapisti taklit eden bir programdır.')
    print('Orijinal program, Joseph Weizenbaum tarafından 1966 yılında tanımlanmıştır.')
    print('Benimle sohbet edebilirsiniz. Çıkmak için "Çıkış" yazabilirsiniz.')
    print('='*72)
    print('Merhaba! Bugün nasıl hissediyorsunuz?')

    s = ''
    therapist = Eliza()
    while s.lower() not in ['çıkış', 'cikis', 'çık', 'cik']:
        try:
            s = input('> ')
        except EOFError:
            s = 'çıkış'
        while s and s[-1] in '!.':
            s = s[:-1]
        print(therapist.respond(s))

if __name__ == "__main__":
    command_interface()



Turkish ELIZA
---------
Terapi Botu
---------
ELIZA, Rogerian tarzda bir psikoterapisti taklit eden bir programdır.
Orijinal program, Joseph Weizenbaum tarafından 1966 yılında tanımlanmıştır.
Benimle sohbet edebilirsiniz. Çıkmak için "Çıkış" yazabilirsiniz.
Merhaba! Bugün nasıl hissediyorsunuz?
